In [2]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_excel("./data/1_train.xlsx")
data.head(5)

,主页ID,主页名称,分类,message,name,description,发文类型,y
0,155000000000000,纬来体育台官方粉丝团,电视频道,中华队长林智胜 三分砲！！！\n人如其名 真的「致胜」阿～,NaN,NaN,video,H
1,359000000000,公视粉丝团,电视频道,您所选择的电视频道，将会决定您的未来~\n遥控器的决定权，就在你手上！\n\n#看见更好的未来,2015公视电视募款 公视让你看见更好的未来,这是一个资讯爆炸的时代，你所选择的资讯将决定你的视野；你所选择的电视频道，将会决定你的未来 ...,video,L
2,359000000000,公视粉丝团,电视频道,好演员值得更多肯定和掌声！\n\n吴慷仁提到大概有六、七年资历的演员，所面临的最大困境，是赚...,公视粉丝团,NaN,link,L
3,190000000000,Mobile01,网路社群,大师兄 一发炸裂！！帅啊啊啊啊啊啊啊！！！,来自 Mobile01 贴文的相片,NaN,photo,H
4,359000000000,公视粉丝团,电视频道,「再不救，就来不及了！如果没有很强力的救，再三十年，你可能找不到鹿港的味道了！」\n\n「两...,【我们的岛】抢救鹿港,叶明兰、叶镇中 / 采访报导 「再不救，就来不及了！如果没有很强力的救，再三十年，你可能找不...,link,L


In [ ]:
grp = data.pivot_table(index='主页ID', columns='y', values=['分类'], aggfunc='count').droplevel(0, axis=1)
grp = grp.fillna(0)
blacklist = grp[ ((grp['H'] <= 1) & (grp['L'] >5)) ]
whitelist = grp[ (grp['L'] == 0) & (grp['H'] > 1) ]
data_remove_bw = data[ (~data['主页ID'].isin(blacklist.index.values)) &
                       (~data['主页ID'].isin(whitelist.index.values)) ]
data_remove_bw = data_remove_bw.reset_index()

In [ ]:
counts50 = grp[(grp['H'] + grp['L'])>=50]
counts50['hl_score'] = np.log2(counts50['H'] + 0.1) / np.log2(counts50['H'] + counts50['L'] + 0.1)
data_tmp = pd.merge(data_remove_bw, counts50, how='left', on='主页ID')
data_hl_score = data_tmp['hl_score'].fillna(0)
data_hl_score

In [1]:
from Python.pandas.feature_selection import *
fl_mapping = {'电视频道': 0, '网路社群': 1, '新闻': 2, '书籍杂志': 3, '电视剧': 4, '电影公司': 5, " \
             "'节目': 6, '媒体其他': 7, '电影': 5, '音乐': 7, '电影相关活动': 5, '广播': 7}
data_fl, fl_ohe = one_hot_encoder_column(data_remove_bw, '分类', fl_mapping, fill_na=7)
data_fl = pd.DataFrame(data_fl)
data_fl

NameError: name 'data_remove_bw' is not defined

In [ ]:
doc_type_map = {'link': 1, 'photo': 2, 'video': 3}
data_doc_type, ohe_doc_type = one_hot_encoder_column(data_remove_bw, '发文类型', doc_type_map, fill_na=3)
data_doc_type

In [ ]:
from Python.nlp.nlp import *
stop_words = get_stop_words('./data/stop_words.txt', encoding='utf-8')
data_all = data.apply(lambda x: split_word(
    str(x['message']) + str(x['name']) + str(x['description']),
    stop_words, ['蔡依林', '叶明兰','叶镇中', '宋家豪', '白先勇','林智胜']),
                      axis=1)

data_all.to_csv("data/lines.csv")

In [9]:
data['nosplit'] = data.apply(lambda x: str(x['message']) + str(x['name']) + str(x['description']), axis=1)
#data_nosplit.to_csv("data/no_lines.csv")

In [7]:
from ltp import LTP
ltp = LTP(path="/Users/proffl/nltk_data/small")

In [8]:
seg, hidden = ltp.seg(["中华队长林智胜 三分砲！！！\n人如其名\u3000真的「致胜」阿～nanna。"])
pos = ltp.pos(hidden) # 词性标注
ner = ltp.ner(hidden) # 命名实体识别
srl = ltp.srl(hidden) # 语义角色标注
dep = ltp.dep(hidden) # 依存句法分析
sdp = ltp.sdp(hidden) # 语义依存分析(树)
# sdp = ltp.sdp(hidden, mode='graph')  # 语义依存分析(图)
print(seg)
print(pos)
print(ner)

[['中华队长', '林智胜', '三分砲', '！', '！', '！', '人如其名\u3000真的', '「', '致胜', '」', '阿～nanna', '。']]
[['ni', 'nh', 'n', 'wp', 'wp', 'wp', 'i', 'wp', 'v', 'wp', 'nh', 'wp']]
[[('Ni', 0, 0), ('Nh', 1, 1), ('Nh', 10, 10)]]


In [ ]:
seg, hidden = ltp.seg(data['nosplit'].tolist())

In [6]:
all_words = []
for doc in data['nosplit'].tolist():
    seg, hidden = ltp.seg([doc])
    pos = ltp.pos(hidden)[0]
    words = []
    for (idx, p) in enumerate(pos):
        if p.startswith('n'):
            words.append(seg[0][idx])
    all_words.append(words)
all_words[:5]

KeyError: 'nosplit'

In [ ]:
data['nosplit'].tolist()[0]

In [ ]:
data_nosplit = data.apply(lambda x: str(x['message']) + str(x['name']) + str(x['description']), axis=1)

In [ ]:
data_nosplit = data_nosplit.tolist()
data_nosplit

In [ ]:
seg, hidden = ltp.seg(data_nosplit[:10])
ner = ltp.ner(hidden)
ner

In [ ]:
keys = set()
for doc in data_nosplit:
    doc_split = doc.split("#")
    if len(doc_split)> 1:
        for s in doc_split[1:]:
            if len(s.strip()) < 8:
                keys.add(s)

In [ ]:
data_all = data.apply(lambda x: split_word(
    str(x['message']) + str(x['name']) + str(x['description']),
    stop_words, keys),axis=1)
data_all